In [52]:
import pandas as pd
from matplotlib import pyplot as plt
from glob import glob
import os.path
from pathlib import Path
import json
import re

# Loading Model Checking Data

In [53]:
# TODO: all paths extract to global const file
deps_synt_hoa_files = glob("../assets/model-checking-results/synt-with-deps/*.hoa")

def deps_synt_analysis(benchmark_name):
    hoa_file_path = "../assets/model-checking-results/synt-with-deps/{}.hoa".format(benchmark_name)
    json_file_path = "../assets/model-checking-results/synt-with-deps/{}.json".format(benchmark_name)
    hoa_file = Path(hoa_file_path).read_text()
    
    # If JSON file doesn't exist, it we had error in synthesis
    if not os.path.exists(json_file_path):
        status = 'error'
        error = ','.join(re.findall('Runtime error:(.*)', hoa_file))
        
        return {
            'name': benchmark_name,
            'status': 'error',
            'error': error
        }
    
    measures = json.loads(Path(json_file_path).read_text())
    
    duration = measures['total_time']
    status = 'completed' if measures['is_completed'] == True  else 'timeout'
    if status != 'completed':
        return {
            'name': benchmark_name,
            'status': status,
            'duration': duration,
        }
    
    total_dependents = sum((1 for v in measures['tested_variables'] if v['is_dependent']))
    total_outputs = len(measures['output_vars'])
    dependents_ratio = total_dependents / total_outputs
    realizablility = measures['synthesis']['independent_strategy']['realizability']
    model_checking = measures['synthesis']['model_checking_status']
    
    return {
        'name': benchmark_name,
        'status': status,
        'duration': duration,
        'realizablility': realizablility,
        'model_checking': model_checking,
        'dependents_ratio': dependents_ratio,
        'total_dependents': total_dependents
    }

deps_synt_df = pd.DataFrame([
    deps_synt_analysis(Path(benchmark_path).stem)
    for benchmark_path in deps_synt_hoa_files
])

deps_synt_df

,name,status,duration,realizablility,model_checking,dependents_ratio,total_dependents,error
0,TwoCountersInRangeA4,completed,113.0,UNREALIZABLE,UNKNOWN,0.375000,3.0,NaN
1,ltl2dba05,completed,7.0,REALIZABLE,OK,0.000000,0.0,NaN
2,ltl2dba11,completed,3.0,REALIZABLE,OK,0.000000,0.0,NaN
3,TwoCountersDisButA6,completed,10747.0,UNREALIZABLE,UNKNOWN,0.307692,4.0,NaN
4,TorcsSteeringSimple,error,NaN,NaN,NaN,NaN,NaN,Unable to open ./tmp/TorcsSteeringSimple.aag
...,...,...,...,...,...,...,...,...
202,OneCounterGuiA1,completed,38.0,UNREALIZABLE,UNKNOWN,0.333333,3.0,NaN
203,EscalatorSmart,error,NaN,NaN,NaN,NaN,NaN,Unable to open ./tmp/EscalatorSmart.aag
204,OneCounterGuiA0,completed,19.0,UNREALIZABLE,UNKNOWN,0.333333,3.0,NaN
205,ltl2dba20,completed,12.0,REALIZABLE,OK,0.000000,0.0,NaN


In [54]:
# TODO: all paths extract to global const file
withouts_deps_synt_hoa_files = glob("../assets/model-checking-results/synt-without-deps/*.hoa")

def without_deps_synt_analysis(benchmark_name):
    hoa_file_path = "../assets/model-checking-results/synt-without-deps/{}.hoa".format(benchmark_name)
    json_file_path = "../assets/model-checking-results/synt-without-deps/{}.json".format(benchmark_name)
    hoa_file = Path(hoa_file_path).read_text()
    
    # If JSON file doesn't exist, it we had error in synthesis
    if not os.path.exists(json_file_path):
        status = 'error'
        error = ','.join(re.findall('Runtime error:(.*)', hoa_file))
        
        return {
            'name': benchmark_name,
            'status': 'error',
            'error': error
        }
    
    measures = json.loads(Path(json_file_path).read_text())
    
    duration = measures['total_time']
    status = 'completed' if measures['is_completed'] == True  else 'timeout'
    if status != 'completed':
        return {
            'name': benchmark_name,
            'status': status,
            'duration': duration,
        }
    
    total_dependents = sum((1 for v in measures['tested_variables'] if v['is_dependent']))
    total_outputs = len(measures['output_vars'])
    dependents_ratio = total_dependents / total_outputs
    realizablility = measures['synthesis']['independent_strategy']['realizability']
    model_checking = measures['synthesis']['model_checking_status']
    
    return {
        'name': benchmark_name,
        'status': status,
        'duration': duration,
        'realizablility': realizablility,
        'model_checking': model_checking,
        'dependents_ratio': dependents_ratio,
        'total_dependents': total_dependents
    }

without_deps_synt_df = pd.DataFrame([
    without_deps_synt_analysis(Path(benchmark_path).stem)
    for benchmark_path in withouts_deps_synt_hoa_files
])

without_deps_synt_df

,name,status,duration,realizablility,model_checking,dependents_ratio,total_dependents,error
0,TwoCountersInRangeA4,completed,425.0,UNREALIZABLE,UNKNOWN,0.0,0.0,NaN
1,ltl2dba05,completed,8.0,REALIZABLE,OK,0.0,0.0,NaN
2,ltl2dba11,completed,3.0,REALIZABLE,OK,0.0,0.0,NaN
3,TwoCountersDisButA6,completed,27194.0,UNREALIZABLE,UNKNOWN,0.0,0.0,NaN
4,TorcsSteeringSimple,completed,19.0,REALIZABLE,OK,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...
202,OneCounterGuiA1,completed,74.0,UNREALIZABLE,UNKNOWN,0.0,0.0,NaN
203,EscalatorSmart,completed,70.0,REALIZABLE,OK,0.0,0.0,NaN
204,OneCounterGuiA0,completed,43.0,UNREALIZABLE,UNKNOWN,0.0,0.0,NaN
205,ltl2dba20,completed,12.0,REALIZABLE,OK,0.0,0.0,NaN


In [55]:
def seconds_to_ms(secs):
    return secs * 1000

TIMEOUT_MS = seconds_to_ms(60*39) # 39 minutes

# TODO: all paths extract to global const file
spot_sd_hoa_files = glob("../assets/model-checking-results/spot-sd/*.hoa")

def spot_sd_analysis(benchmark_name):
    hoa_file_path = "../assets/model-checking-results/spot-sd/{}.hoa".format(benchmark_name)
    hoa_file = Path(hoa_file_path).read_text()
    
    duration_str = re.findall('real\s+(.*)m(.*)s', hoa_file)
    if len(duration_str) != 1:
        raise Exception(f'Duration not found for benchmark {benchmark_name}, duration = ', duration)

    duration_minutes = float(duration_str[0][0])
    duration_seconds = float(duration_str[0][1])
    duration = seconds_to_ms(duration_seconds + (60*duration_minutes))
    
    is_completed = ('UNREALIZABLE' in hoa_file) or ('REALIZABLE' in hoa_file)
    if not is_completed:
        status = 'timeout' if duration >= TIMEOUT_MS else 'error'
        
        if status == 'error':
            error = ', '.join(re.findall('ltlsynt:(.*)', hoa_file))
        else:
            error = None
        
        return {
            'name': benchmark_name,
            'status': status,
            'error': error,
            'duration': duration
        }
    
    realizablility = 'UNREALIZABLE' if 'UNREALIZABLE' in hoa_file else 'REALIZABLE'
    
    if 'Circuit was verified' in hoa_file:
        model_checking = 'OK'
    elif 'circuit is not OK.' in hoa_file:
        model_checking = 'ERROR'
    else:
        model_checking = None
    
    return {
        'name': benchmark_name,
        'status': 'completed',
        'duration': duration,
        'model_checking': model_checking,
        'realizablility': realizablility
    }

spot_sd_df = pd.DataFrame([
    spot_sd_analysis(Path(benchmark_path).stem)
    for benchmark_path in spot_sd_hoa_files
])

spot_sd_df

,name,status,duration,model_checking,realizablility,error
0,TwoCountersInRangeA4,completed,3161.0,None,UNREALIZABLE,NaN
1,ltl2dba05,completed,69.0,OK,REALIZABLE,NaN
2,ltl2dba11,completed,28.0,OK,REALIZABLE,NaN
3,TwoCountersDisButA6,timeout,2402700.0,NaN,NaN,None
4,TorcsSteeringSimple,completed,42.0,OK,REALIZABLE,NaN
...,...,...,...,...,...,...
202,OneCounterGuiA1,completed,128.0,None,UNREALIZABLE,NaN
203,EscalatorSmart,completed,58.0,OK,REALIZABLE,NaN
204,OneCounterGuiA0,completed,103.0,None,UNREALIZABLE,NaN
205,ltl2dba20,completed,39.0,OK,REALIZABLE,NaN


# Synthesis Model Checking Resutls

In [78]:
X = deps_synt_df[["name", "model_checking","realizablility","status","error"]]
X[(X["realizablility"] != 'UNREALIZABLE') & (X["model_checking"] != 'OK')]

,name,model_checking,realizablility,status,error
4,TorcsSteeringSimple,NaN,NaN,error,Unable to open ./tmp/TorcsSteeringSimple.aag
5,SensorRegister,Error,REALIZABLE,completed,NaN
16,EscalatorNonCounting,Error,REALIZABLE,completed,NaN
17,Zoo5,NaN,NaN,error,Unable to open ./tmp/Zoo5.aag
18,Increment,Error,REALIZABLE,completed,NaN
...,...,...,...,...,...
192,Radarboard,Error,REALIZABLE,completed,NaN
193,ModdifiedLedMatrix5X,NaN,NaN,error,Unable to open ./tmp/ModdifiedLedMatrix5X.aag
195,TwoCounters4,NaN,NaN,error,Unable to open ./tmp/TwoCounters4.aag
201,MusicAppMotivating,NaN,NaN,error,Unable to open ./tmp/MusicAppMotivating.aag


In [90]:
X = deps_synt_df[["name", "model_checking","realizablility","status","error"]]
deps_synt_err_instances = X[(X["realizablility"] != 'UNREALIZABLE') & (X["model_checking"] != 'OK') & (X["status"] == 'completed')]
deps_synt_err_instances

,name,model_checking,realizablility,status,error
5,SensorRegister,Error,REALIZABLE,completed,NaN
16,EscalatorNonCounting,Error,REALIZABLE,completed,NaN
18,Increment,Error,REALIZABLE,completed,NaN
45,Button,Error,REALIZABLE,completed,NaN
103,Scoreboard,Error,REALIZABLE,completed,NaN
105,KitchenTimerV0,Error,REALIZABLE,completed,NaN
121,SPIReadClk,Error,REALIZABLE,completed,NaN
126,ActionConverter,Error,REALIZABLE,completed,NaN
143,RegManager,Error,REALIZABLE,completed,NaN
166,SPIWriteClk,Error,REALIZABLE,completed,NaN


In [94]:

X = without_deps_synt_df[["name", "model_checking","realizablility","status","error"]]
pd.merge(deps_synt_err_instances, X, on="name", suffixes=('_deps_synt', '_without_deps_synt'))
# X[(X["realizablility"] != 'UNREALIZABLE') & (X["model_checking"] != 'OK') & (X["status"] == 'completed')]

,name,model_checking_deps_synt,realizablility_deps_synt,status_deps_synt,error_deps_synt,model_checking_without_deps_synt,realizablility_without_deps_synt,status_without_deps_synt,error_without_deps_synt
0,SensorRegister,Error,REALIZABLE,completed,NaN,OK,REALIZABLE,completed,NaN
1,EscalatorNonCounting,Error,REALIZABLE,completed,NaN,OK,REALIZABLE,completed,NaN
2,Increment,Error,REALIZABLE,completed,NaN,OK,REALIZABLE,completed,NaN
3,Button,Error,REALIZABLE,completed,NaN,OK,REALIZABLE,completed,NaN
4,Scoreboard,Error,REALIZABLE,completed,NaN,OK,REALIZABLE,completed,NaN
5,KitchenTimerV0,Error,REALIZABLE,completed,NaN,OK,REALIZABLE,completed,NaN
6,SPIReadClk,Error,REALIZABLE,completed,NaN,OK,REALIZABLE,completed,NaN
7,ActionConverter,Error,REALIZABLE,completed,NaN,OK,REALIZABLE,completed,NaN
8,RegManager,Error,REALIZABLE,completed,NaN,OK,REALIZABLE,completed,NaN
9,SPIWriteClk,Error,REALIZABLE,completed,NaN,OK,REALIZABLE,completed,NaN


In [77]:
X = deps_synt_df[["name", "model_checking","realizablility","status","error"]]
X[X["model_checking"] == 'OK']

,name,model_checking,realizablility,status,error
1,ltl2dba05,OK,REALIZABLE,completed,NaN
2,ltl2dba11,OK,REALIZABLE,completed,NaN
7,ltl2dba10,OK,REALIZABLE,completed,NaN
8,ltl2dba04,OK,REALIZABLE,completed,NaN
10,ltl2dba12,OK,REALIZABLE,completed,NaN
11,ltl2dba06,OK,REALIZABLE,completed,NaN
15,amba_decomposed_tsingle,OK,REALIZABLE,completed,NaN
23,ltl2dba07,OK,REALIZABLE,completed,NaN
24,ltl2dba13,OK,REALIZABLE,completed,NaN
29,ltl2dba17,OK,REALIZABLE,completed,NaN


# Join Model Checking

In [56]:
A = pd.merge(spot_sd_df[["name", "model_checking"]], deps_synt_df[["name", "model_checking"]], on="name", suffixes=('_spot_sd', '_deps_synt'))
model_checking_df = pd.merge(A, without_deps_synt_df[["name", "model_checking"]], on="name")
model_checking_df.rename(columns={"model_checking": "model_checking_without_deps_synt"}, inplace=True)
model_checking_df

,name,model_checking_spot_sd,model_checking_deps_synt,model_checking_without_deps_synt
0,TwoCountersInRangeA4,None,UNKNOWN,UNKNOWN
1,ltl2dba05,OK,OK,OK
2,ltl2dba11,OK,OK,OK
3,TwoCountersDisButA6,NaN,UNKNOWN,UNKNOWN
4,TorcsSteeringSimple,OK,NaN,OK
...,...,...,...,...
202,OneCounterGuiA1,None,UNKNOWN,UNKNOWN
203,EscalatorSmart,OK,NaN,OK
204,OneCounterGuiA0,None,UNKNOWN,UNKNOWN
205,ltl2dba20,OK,OK,OK
